In [4]:
import fitz  # PyMuPDF

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

def clean_text(text):
    import re
    text = text.replace("\n", " ")
    text = re.sub(r'\s+', ' ', text)  # Mehrfache Leerzeichen entfernen
    return text.strip()

def chunk_text(text, max_length=500):
    import re
    sentences = re.split(r'(?<=[.!?])\s+', text)
    chunks, current_chunk = [], ""
    
    for sentence in sentences:
        if len(current_chunk) + len(sentence) <= max_length:
            current_chunk += " " + sentence
        else:
            chunks.append(current_chunk.strip())
            current_chunk = sentence
    if current_chunk:
        chunks.append(current_chunk.strip())
    
    return chunks

raw_text = extract_text_from_pdf("2023_Volkswagen_Group_Nachhaltigkeitsbericht.pdf")
text = clean_text(raw_text)
chunks = chunk_text(text)

# Text in cunks unterteilen

In [2]:
print(chunks)

['', '2023 N A C H H A LT I G K E I T S B E R I C H T Inhalt Navigationshilfe Kapitelverweise innerhalb des Nachhaltigkeitsberichts Verweise auf Websites 6 Vorwort 9 Über den Bericht 12 \x07 Nachhaltigkeit in der Unternehmens-DNA 13 Corporate Governance 17 Konzernstrategie NEW AUTO 22 \x07ESG-Performance-Management und Wesentlichkeitsanalyse 25 Nachhaltigkeitsmanagement 27 \x07Nachhaltigkeitswirkungen neuer Mobilitätsangebote 30 Stakeholder-Management 33 Umwelt-Compliance-Management 42 Risikomanagement 45 Corporate Citizenship Fokusthemen 1 \x07Dekarbonisierung 48 Dekarbonisierung 69 EU-Taxonomie 2 \x07Kreislaufwirtschaft 83 Kreislaufwirtschaft 3 \x07Menschen in der Transformation 90 Menschen in der Transformation 4 \x07Vielfalt 104 Vielfalt 5 \x07Integrität und Compliance 109 Integrität und Compliance 6 \x07Lieferkette und Menschenrechte 117 Lieferkette und Menschenrechte 131 Anhang 132 \x07Vermerk des unabhängigen Wirtschaftsprüfers 135 Impressum 2 Nachhaltigkeits\xad kommunikation d

# KeyBERT zur Extraktion der relevantesten Aussagen

In [8]:
from keybert import KeyBERT
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("paraphrase-MiniLM-L6-v2")
kw_model = KeyBERT(model=model)

keywords_per_chunk = [
    (chunk, kw_model.extract_keywords(chunk, keyphrase_ngram_range=(1, 3), stop_words='german', top_n=3))
    for chunk in chunks
]


In [6]:
for i, p in enumerate(keywords_per_chunk, 1):
    print(f"{i}. {p}\n")

1. ('', [])

2. ('2023 N A C H H A LT I G K E I T S B E R I C H T Inhalt Navigationshilfe Kapitelverweise innerhalb des Nachhaltigkeitsberichts Verweise auf Websites 6 Vorwort 9 Über den Bericht 12 \x07 Nachhaltigkeit in der Unternehmens-DNA 13 Corporate Governance 17 Konzernstrategie NEW AUTO 22 \x07ESG-Performance-Management und Wesentlichkeitsanalyse 25 Nachhaltigkeitsmanagement 27 \x07Nachhaltigkeitswirkungen neuer Mobilitätsangebote 30 Stakeholder-Management 33 Umwelt-Compliance-Management 42 Risikomanagement 45 Corporate Citizenship Fokusthemen 1 \x07Dekarbonisierung 48 Dekarbonisierung 69 EU-Taxonomie 2 \x07Kreislaufwirtschaft 83 Kreislaufwirtschaft 3 \x07Menschen in der Transformation 90 Menschen in der Transformation 4 \x07Vielfalt 104 Vielfalt 5 \x07Integrität und Compliance 109 Integrität und Compliance 6 \x07Lieferkette und Menschenrechte 117 Lieferkette und Menschenrechte 131 Anhang 132 \x07Vermerk des unabhängigen Wirtschaftsprüfers 135 Impressum 2 Nachhaltigkeits\xad kom

# Nachhaltigkeitsversprechen herausfiltern

In [10]:
relevant_keywords = [
    #"verpflichtet", "verpflichten", "nachhaltigkeit","ziel","strategie",, 
    "verantwortung", "klimaneutral",  
    "2030", "klimaschutz", "emissionen",  "deklarieren", "klimaziele", 
    "co2", "paris", "klimaneutralität", "umweltziele"
]

def is_sustainability_promise_v2(chunk):
    return any(term.lower() in chunk.lower() for term in relevant_keywords)

filtered_chunks = [chunk for chunk in chunks if is_sustainability_promise_v2(chunk)]


# Ausgabe
for i, p in enumerate(filtered_chunks, 1):
    print(f"{i}. {p}\n")


1. Nachhaltigkeit in der Unter­ nehmens-­DNA Dekarbonisierung Kreislauf­ wirtschaft Menschen in der Transformation Vielfalt Integrität und Compliance Lieferkette und Menschenrechte Progressive Core Sport Luxury Trucks Architecture Software Battery, Charging & Energy Volkswagen Group Mobility 06 | ESG, Decarbonization and Integrity Highlights 20231 Dekarbonisierung Zugang zu 600.000 Ladepunkten in Europa Bis 2030 soll der CO₂-Fußabdruck des gesamten Handelsnetzes um mindestens 30 % gesenkt werden.

2. Für unsere Belegschaft, unsere Kunden, unsere Investoren – und für ­unsere Gesellschaft im Ganzen. Als einer der weltweit größten Industriekonzerne tragen wir eine besondere Verantwortung. Wir können einen Unterschied machen. Diese Chance wollen und werden wir nutzen. Wir denken Nachhaltigkeit ganzheitlich – im Sinne der Natur, der Menschen, der Gesellschaft und des wertschaffenden Unternehmertums. Von der Lieferkette über unsere Produktion bis hin zur Auslieferung und Nutzung unser

In [20]:
pip install sentencepiece

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 1.6 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [20]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("paraphrase-MiniLM-L6-v2")
kw_model = KeyBERT(model=model)

zusammenfassungen = []
for chunk in filtered_chunks:
    keywords = kw_model.extract_keywords(chunk, keyphrase_ngram_range=(1, 10), stop_words=None, top_n=10)
    print(keywords)
    zusammenfassungen.append(keywords)
    

for kw in keywords:
    print(f"- {kw[0]} (Score: {kw[1]:.3f})")

[('compliance lieferkette und menschenrechte progressive core sport luxury trucks architecture', 0.6677), ('und compliance lieferkette und menschenrechte progressive core sport luxury trucks', 0.6645), ('compliance lieferkette und menschenrechte progressive core sport luxury trucks', 0.6412), ('lieferkette und menschenrechte progressive core sport luxury trucks architecture', 0.6277), ('und menschenrechte progressive core sport luxury trucks architecture software battery', 0.6276), ('lieferkette und menschenrechte progressive core sport luxury trucks architecture software', 0.6262), ('und menschenrechte progressive core sport luxury trucks architecture', 0.6125), ('lieferkette und menschenrechte progressive core sport luxury trucks', 0.6002), ('menschenrechte progressive core sport luxury trucks architecture software battery', 0.5968), ('und menschenrechte progressive core sport luxury trucks architecture software', 0.5905)]
[('unsere investoren und für unsere gesellschaft im ganzen al

KeyboardInterrupt: 

In [22]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

model_name = "facebook/bart-large-cnn"

# Modell und Tokenizer laden
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
summarizer = pipeline("summarization", model=model, tokenizer=tokenizer)

# Zusammenfassungen erzeugen
zusammenfassungen = []
for chunk in filtered_chunks:
    summary = summarizer(chunk, max_length=80, min_length=30, do_sample=False)[0]["summary_text"]
    print(summary)
    zusammenfassungen.append(summary)

finale_zusammenfassung = " ".join(zusammenfassungen)

# Ausgabe
print("Einzelne Zusammenfassungen:")
for i, z in enumerate(zusammenfassungen, 1):
    print(f"\nChunk {i}:\n{z}")

print("\n--- Finale Gesamtzusammenfassung ---\n")
print(finale_zusammenfassung)

Device set to use cpu


Bis 2030 soll der CO₂-Fußabdruck des gesamten Handelsnetzes um mindestens 30 % gesenkt werden. Volkswagen Group Mobility 06 | ESG, Decarbonization and Integrity Highlights 20231.
"Wir denken Nachhaltigkeit ganzheitlich – im Sinne der Natur, der Menschen, der Gesellschaft and des wertschaffenden Unternehmertums" "Wir können einen Unterschied machen. Diese Chance wollen und werden wir
Unser Ziel ist es, die Natur und die Gesellschaft positiv mitzugestal­ ten. Das ist ambitioniert. Und es wird uns Kraftanstrengungen abverlangen. Zugleich ist das Erreichen dieses Ziels eine Verpflichtung. Nachhalt
Volkswagen ist in der persönlichen Verantwortung des Managements, diese einzuhalten. Alle Ziele werden systematisch auf die Marken und Konzern­ gesellschaften heruntergebrochen.
Volkswagen: Die produktionsbedingten ­CO2-Emissionen von Pkw und leichten Nutzfahrzeugen bis 2030 um 50,4 Prozent im Vergleich zum Basisjahr 2018 zu reduce.


KeyboardInterrupt: 

---------------------------------------------------------------------------------------------------------------------

In [16]:
print("Einzelne Zusammenfassungen:")
for i, z in enumerate(zusammenfassungen, 1):
    print(f"\nChunk {i}:\n{z}")

Einzelne Zusammenfassungen:

Chunk 1:
Nachhaltigkeitswirkungen neuer Mobilitätsangebote 30 Stakeholder-Management 33 Umwelt-Compliance-Management 42 Risikomanagement 45 Corporate Citizenship Fokusthemen 1   Dekarbonisierung 48 Dekarbonisie 69 EU-Taxonomie 2   Kreislaufwirtschaft 83 KreislauFWirtschafts 3   Menschen in der Transformation

Chunk 2:
Konzernstrategie NEW AUTO stellen uns zukunftsgerichtet auf. Dabei behalten wir unser Ziel, ein weltweit führender Anbieter nachhaltiger Mobilität. richten den Konzern fokussierter, effizienter, innovativer.

Chunk 3:
Konzernstrategie NEW AUTO definiert für den Bereich Nachhaltigkeit und ESG in der Konzerninitiative 6 die sechs oben ab­gebil­deten wesentlichen Fokusthemen. Nach diesen ist der vorliegende Nach hartigkeitsbericht inklusive nichtfinanzieller Bericht.

Chunk 4:
Ein Einblick in Themen wie Risikomanagement, Umweltmanagement, Stakeholder-Management und Corporate Citizenship. Eine Definition von Nachhaltigkeit für das Unternehmen.

Ch

In [9]:
pip install torch==2.6.0

ERROR: Could not find a version that satisfies the requirement torch==2.6.0 (from versions: 1.11.0, 1.12.0, 1.12.1, 1.13.0, 1.13.1, 2.0.0, 2.0.1, 2.1.0, 2.1.1, 2.1.2, 2.2.0, 2.2.1, 2.2.2)
ERROR: No matching distribution found for torch==2.6.0
Note: you may need to restart the kernel to use updated packages.


In [7]:
import torch
print(torch.__version__)

2.2.2


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from tqdm import tqdm
import pandas as pd

# 📌 Deutsches Zusammenfassungsmodell laden
model_name = "mrm8488/bert2bert_shared-german-finetuned-summarization"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
summarizer = pipeline("summarization", model=model, tokenizer=tokenizer)

# 🧠 Funktion zur Key-Fact-Kompression
def compress_keyfact(text, max_len=50):
    try:
        result = summarizer(text, max_length=max_len, min_length=20, do_sample=False)
        return result[0]['summary_text']
    except Exception as e:
        print(f"Fehler bei Chunk: {e}")
        return None

# 🎯 Deine bereits gefilterten Chunks
# Beispiel: filtered_chunks = ["Die renommierte Science Based Targets Initiative ...", ...]
keyfacts = []

print("🔍 Extrahiere Key-Facts ...")
for chunk in tqdm(filtered_chunks):
    if len(chunk.strip()) > 50:
        summary = compress_keyfact(chunk)
        if summary:
            keyfacts.append({"original": chunk, "keyfact": summary})

# 💾 Ergebnisse als CSV speichern
df = pd.DataFrame(keyfacts)
df.to_csv("keyfacts_extrahiert.csv", index=False, encoding="utf-8-sig")

print("✅ Key-Facts extrahiert und gespeichert in 'keyfacts_extrahiert.csv'")


The following encoder weights were not tied to the decoder ['bert/pooler']
The following encoder weights were not tied to the decoder ['bert/pooler']
The following encoder weights were not tied to the decoder ['bert/pooler']
Device set to use cpu


NotImplementedError: Cannot copy out of meta tensor; no data!

# BERTopic für Themenanalyse

In [6]:
from bertopic import BERTopic

topic_model = BERTopic(language="german", embedding_model=model)
topics, probs = topic_model.fit_transform(chunks)

# Anzeigen der wichtigsten Themen
topic_model.get_topic_info()


,Topic,Count,Name,Representation,Representative_Docs
0,0,1057,0_und_der_die_in,"[und, der, die, in, im, für, von, den, zu, vol...",[www.volkswagen-group.com > Charta der ­Beruf...
1,1,117,1_und_in_der_vielfalt,"[und, in, der, vielfalt, compliance, menschenr...",[Inhalt Unternehmens-DNA | Dekarbonisierung |...
